In [2]:
import numpy as np

In [3]:
import torch
import torch.nn as nn
import math
from torch.autograd import Variable

In [4]:
import torch.nn.functional as F

# Embedding 

In [5]:
class Embedder(nn.Module):
    def __init__(self, vocab_size:int, emb_dim:int):
        super().__init__()
        self.emb_dim = emb_dim 
        self.embed = nn.Embedding(vocab_size, emb_dim)

    def forward(self, x: torch.tensor):
        return self.embed(x)

# Why use `register_buffer`

- Ans [link](https://discuss.pytorch.org/t/what-is-the-difference-between-register-buffer-and-register-parameter-of-nn-module/32723/11)

An example where I find this distinction difficult is in 
the context of fixed positional encodings in the Transformer 
model. Typically I see implementations where the fixed positional 
encodings are registered as buffers but I’d consider these tensors 
as non-learnable parameters (that should show up in the list of 
model parameters), especially when comparing between methods 
that don’t rely on such injection of fixed tensors.

So in general:
- buffers = `fixed tensors / non-learnable parameters / stuff that does not require gradient`
- parameters = `learnable parameters, requires gradient`

![image](https://discuss.pytorch.org/user_avatar/discuss.pytorch.org/ptrblck/90/1823_2.png)
Piotr Bialecki

If you have parameters in your model, which should be saved and restored in the state_dict, but not trained by the optimizer, you should register them as buffers. Buffers won’t be returned in model.parameters(), so that the optimizer won’t have a change to update them.

Both approaches work the same regarding training etc.
There are some differences in the function calls however. Using register_parameter you have to pass the name as a string, which can make the creation of a range of parameters convenient. Besides that I think it’s just coding style which one you prefer.

If your `self.some_params` are `nn.Parameter` objects, then you don’t have to worry about this. If they’re tensors, then they won’t be in the `state_dict` (unless registered as buffer).

> simple `torch.tensor` will not be available under `state_dict`

one reason to register the tensor as a buffer is to be able to serialize the model and restore all internal states.
Another one is that all buffers and parameters will be pushed to the device, if called on the parent model:

```python
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.my_tensor = torch.randn(1)
        self.register_buffer('my_buffer', torch.randn(1))
        self.my_param = nn.Parameter(torch.randn(1))
        
    def forward(self, x):
            return x

model = MyModel()
print(model.my_tensor)
> tensor([0.9329])
print(model.state_dict())
> OrderedDict([('my_param', tensor([-0.2471])), ('my_buffer', tensor([1.2112]))])

model.cuda()
print(model.my_tensor)
> tensor([0.9329])
print(model.state_dict())
> OrderedDict([('my_param', tensor([-0.2471], device='cuda:0')), ('my_buffer', tensor([1.2112], device='cuda:0'))])
```

As you can see, model.my_tensor is still on the CPU, where is was created, while all parameters and buffers were pushed to the GPU after calling `model.cuda()`.

<center>
    <img src="https://miro.medium.com/max/566/1*B-VR6R5vJl3Y7jbMNf5Fpw.png" width="400">
</center>

# Make embedding relatively larger by scaling the values. WHY?

The reason we increase the embedding values before 
addition is to make the positional encoding relatively 
smaller. This means the original meaning in the embedding 
vector won’t be lost when we add them together

```python
x = x*math.sqrt(self.emb_dim)
```

In [6]:
class PositionalEmbedding(nn.Module):
    def __init__(self, emb_dim:int, max_seq_len:int = 200, dropout_pct:float = 0.1):
        super().__init__()

        self.emb_dim = emb_dim
        self.dropout = nn.Dropout(dropout_pct)

        # create constant 'pe' matrix with values dependent on 
        # word position 'pos' and embedding position 'i'
        pe = torch.zeros(max_seq_len, emb_dim)

        for pos in range(max_seq_len):
            for i in range(0, emb_dim, 2):
                pe[pos,i] = math.sin(pos/(1000**((2*i)/emb_dim)))
                pe[pos,i+1] = math.sin(pos/(1000**((2*(i+1))/emb_dim)))

        # print(pe.size())
        # pe = pe.unsqueeze(0)
        # print(pe.size())

        self.register_buffer('pe', pe)


    def forward(self, x):
        
        # scale values
        x = x*math.sqrt(self.emb_dim)  

        seq_len = x.size(0)
        
        # add constant positional embedding to the word embedding
        pe = Variable(self.pe[:seq_len,:], requires_grad=False)
        
        if x.is_cuda:
            pe.cuda()
        
        x = x + pe
        return self.dropout(x)

## Test module

In [7]:
VOCAB_SIZE = 20000
EMB_DIM = 512

In [8]:
e = Embedder(VOCAB_SIZE, EMB_DIM)

In [9]:
idx = torch.tensor([1,2,3,4,5]) # torch.randint(3, 5, (3,))

In [10]:
idx

tensor([1, 2, 3, 4, 5])

In [11]:
emb = e(idx)

In [12]:
emb.size()

torch.Size([5, 512])

In [13]:
p = PositionalEmbedding(emb_dim=EMB_DIM)

In [14]:
p.pe.size()

torch.Size([200, 512])

In [15]:
p.pe.squeeze().size()

torch.Size([200, 512])

In [16]:
p_emb = p(emb)

In [17]:
p_emb.size()

torch.Size([5, 512])

# Batch Normalization

Normalisation is highly important in deep neural networks. It prevents the range of values in the layers changing too much, meaning the model trains faster and has better ability to generalise.

<center>
    <img src="https://miro.medium.com/max/511/1*4w3sQ14caDRkrQsAeK5Flw.png" width="400">
</center>

We will be normalising our results between each layer in the encoder/decoder, so before building our model let’s define that function:

- [blog](https://kharshit.github.io/blog/2018/12/28/why-batch-normalization)

<center>
    <img src="https://kharshit.github.io/img/batch_normalization.png" width="400">
</center>

In [18]:
class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()
    
        self.size = d_model
        
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        
        self.eps = eps
    
    def forward(self, x):
        
        x_mean = x.mean(dim=-1, keepdim=True)
        x_variance = x.std(dim=-1, keepdim=True) 
        
        normalized_x = (x - x_mean) / (x_variance + self.eps)
        
        # scale and shift
        y = self.alpha * normalized_x + self.bias
        return y

## Test module

In [19]:
d_model = 5
bs = 2
seq_len = 10
x = torch.rand(size=(bs,seq_len, d_model))
x

tensor([[[0.3549, 0.3643, 0.6217, 0.2935, 0.4385],
         [0.5275, 0.4559, 0.7918, 0.7242, 0.2137],
         [0.0652, 0.1690, 0.8499, 0.0375, 0.5418],
         [0.4561, 0.7032, 0.9282, 0.8615, 0.6306],
         [0.7029, 0.7938, 0.6353, 0.6353, 0.4864],
         [0.0937, 0.5451, 0.6244, 0.8331, 0.3770],
         [0.8098, 0.4133, 0.6112, 0.4413, 0.6418],
         [0.3863, 0.5925, 0.6579, 0.5201, 0.8527],
         [0.7962, 0.4085, 0.9211, 0.4699, 0.0184],
         [0.3266, 0.6983, 0.2370, 0.2909, 0.4459]],

        [[0.3323, 0.7005, 0.1673, 0.8067, 0.7122],
         [0.2645, 0.6763, 0.9156, 0.9988, 0.9627],
         [0.7283, 0.0362, 0.6782, 0.1660, 0.4566],
         [0.0054, 0.6466, 0.1947, 0.4022, 0.3815],
         [0.2907, 0.4536, 0.6271, 0.7440, 0.0601],
         [0.1551, 0.7979, 0.3085, 0.1244, 0.5133],
         [0.5799, 0.5476, 0.6216, 0.6387, 0.4013],
         [0.2493, 0.1290, 0.2412, 0.1952, 0.3215],
         [0.3790, 0.4681, 0.7214, 0.4209, 0.5561],
         [0.1416, 0.3479, 0.8

In [20]:
x.size()

torch.Size([2, 10, 5])

In [21]:
n = Norm(d_model=d_model)

In [22]:
n(x)

tensor([[[-0.4710, -0.3965,  1.6345, -0.9558,  0.1888],
         [-0.0659, -0.3777,  1.0849,  0.7907, -1.4320],
         [-0.7590, -0.4645,  1.4677, -0.8377,  0.5935],
         [-1.3830, -0.0676,  1.1299,  0.7747, -0.4540],
         [ 0.4633,  1.2713, -0.1372, -0.1374, -1.4599],
         [-1.4436,  0.1816,  0.4671,  1.2187, -0.4238],
         [ 1.3998, -1.0525,  0.1713, -0.8793,  0.3608],
         [-1.2486, -0.0542,  0.3241, -0.4736,  1.4522],
         [ 0.7704, -0.3223,  1.1224, -0.1490, -1.4215],
         [-0.3982,  1.6253, -0.8862, -0.5924,  0.2515]],

        [[-0.7616,  0.5641, -1.3556,  0.9467,  0.6064],
         [-1.6304, -0.2853,  0.4965,  0.7686,  0.6506],
         [ 1.0300, -1.2314,  0.8665, -0.8073,  0.1422],
         [-1.3325,  1.3317, -0.5458,  0.3163,  0.2304],
         [-0.5324,  0.0682,  0.7079,  1.1393, -1.3830],
         [-0.8029,  1.4935, -0.2550, -0.9124,  0.4767],
         [ 0.2332, -0.1080,  0.6754,  0.8562, -1.6567],
         [ 0.3099, -1.3807,  0.1962, -0.4506, 

# Attention

<center>
    <img src="https://miro.medium.com/max/445/1*evdACdTOBT5j1g1nXialBg.png" width="400">
</center>


<center>
    <img src="https://miro.medium.com/max/140/1*15E9qKg9bKnWdSRWCyY2iA.png" width="100">
</center>

- Initially we must multiply $Q$ by the transpose of $K$. This is then `scaled` by dividing the output by the square root of $d_k$.
- A step that’s not shown in the equation is the `masking operation`. Before we perform `Softmax`, we apply our mask and hence reduce values where the input is padding (or in the decoder, also where the input is ahead of the current word).

Another step not shown is `dropout`, which we will apply after `Softmax`.

Finally, the last step is doing a `dot` product between the result so far and $V$.

In [23]:
def attention(q, k, v, d_k, mask=None, dropout=None):
    
    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)
    
    if mask is not None:
        mask = mask.unsqueeze(1)
        scores = scores.masked_fill(mask == 0, -1e9)
    
    scores = F.softmax(scores, dim=-1)
    
    if dropout is not None:
        scores = dropout(scores)
        
    output = torch.matmul(scores, v)
    return output

# Multi-Headed Attention

Once we have our embedded values (with positional encodings) and our masks, we can start building the layers of our model.

Here is an overview of the multi-headed attention layer:


<center>
    <img src="https://miro.medium.com/max/523/1*1tsRtfaY9z6HxmERYhw8XQ.png" width="300">
</center>

- $V$, $K$ and $Q$ stand for `key`, `value` and `query`. These are terms used in attention functions

- In the case of the **Encoder**, $V, K$ and $G$ will simply be **identical copies** of the `emb_vector + pos_encoding`. 
- They will have the dimensions `Batch_size * seq_len * d_model`

<center>
    <img src="images/tensor_dimension.png" width="400">
</center>

- In multi-head attention we `split the embedding vector` into `N heads`. $d_{model}^{new} = \frac{d_{model}}{N}$. so they will then have the dimensions `batch_size * N * seq_len * (d_model / N)`.
- This final dimension `(d_model / N )` we will refer to as `d_k`

- Drawing tool [Excalidraw](https://excalidraw.com/)

In [24]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    def forward(self, q, k, v, mask=None):
        
        bs = q.size(0)
        
        # perform linear operation and split into N heads
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        # transpose to get dimensions bs * N * sl * d_model
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)
        

        # calculate attention using function we will define next
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous()\
        .view(bs, -1, self.d_model)
        output = self.out(concat)
    
        return output

## Test module

In [25]:
d_model = 16
heads = 4
bs = 2
seq_len = 10

In [26]:
mha = MultiHeadAttention(heads=heads, d_model=d_model)

In [27]:
q = torch.rand(size=(bs,seq_len, d_model))
k = torch.rand(size=(bs,seq_len, d_model))
v = torch.rand(size=(bs,seq_len, d_model))

In [28]:
output = mha(q,k,v)
output.size() # batch_size x sequence_length x model_dimension 

torch.Size([2, 10, 16])

## Feedforward Network

In [29]:
class FeedForward(nn.Module):
    def __init__(self, d_model:int, d_ff:int = 2048, dropout_pct:float = 0.1):
        super().__init__()
        
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout_pct)
        
    def forward(self, x):
        x = self.dropout(F.relu(self.linear1(x)))
        x = self.linear2(x)
        return x

## Is dropout applied before or after the non-linear activation function?

Typically, dropout is applied after the non-linear activation function (a). However, when using rectified linear units (ReLUs), it might make sense to apply dropout before the non-linear activation (b) for reasons of computational efficiency depending on the particular code implementation.

- (a): Fully connected, `linear activation` -> `ReLU` -> `Dropout` -> … [traditional use]
- (b): Fully connected, `linear activation` -> `Dropout` -> `ReLU` -> …

**Reference:**
- [Sebastian Rachka](https://sebastianraschka.com/faq/docs/dropout-activation.html)

## Test module

In [30]:
d_model = 16
bs = 2
seq_len = 10

In [31]:
x = torch.rand(size=(bs,seq_len, d_model))

In [32]:
ff = FeedForward(d_model=d_model)

In [33]:
output = ff(x)

In [34]:
output.size()

torch.Size([2, 10, 16])


<center>
    <img src="images/attention_layer.jpg" width="700">
</center>

# Basic Encoder Layer

In [70]:
class EncoderLayer_basic(nn.Module):
    def __init__(self, d_model, heads, dropout_pct = 0.1):
        super().__init__()
        
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model, dropout=dropout_pct)
        
        self.ff = FeedForward(d_model, dropout_pct = dropout_pct)
        self.dropout_1 = nn.Dropout(dropout_pct)
        self.dropout_2 = nn.Dropout(dropout_pct)
        
    def forward(self, x, mask):
        # q = k = v = x
        x = x + self.dropout_1(self.attn(x,x,x,mask))
        x = self.norm_1(x)
        
        x = x + self.dropout_2(self.ff(x))
        x = self.norm_2(x)
        
        return x

**Note:** The above implementation creates the `forward` function based on the above encoder image. Which is slightly different from this blog implemention [link](https://github.com/SamLynnEvans/Transformer/blob/master/Layers.py), where their `add` and `norm` operation order is slightly different

```python
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        pass
        
    def forward(self, x, mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x
```

## Test Module

In [73]:
d_model = 16
heads = 4
bs = 2
seq_len = 10

el = EncoderLayer_basic(d_model, heads)

x = torch.rand(size=(bs,seq_len, d_model))

output = el(x,mask=None)
output.size()

# Basic Decoder Layer 

In [74]:
class DecoderLayer_basic(nn.Module):
    def __init__(self, d_model, heads, dropout_pct = 0.1):
        super().__init__()
        
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.norm_3 = Norm(d_model)
        
        self.attn_decoder = MultiHeadAttention(heads, d_model)
        self.attn_encoder = MultiHeadAttention(heads, d_model)
        
        self.ff = FeedForward(d_model)
        
        self.dropout_1 = nn.Dropout(dropout_pct)
        self.dropout_2 = nn.Dropout(dropout_pct)
        self.dropout_3 = nn.Dropout(dropout_pct)
        
    def forward(self, x, encoder_output, src_mask, trg_mask):
        
        """
        x: this x comes from the target language
        """
        
        q = k = v = x
        x = x + self.dropout_1(self.attn_decoder(q,k,v,trg_mask))
        x = self.norm_1(x)
        
        k_enc = v_enc = encoder_output
        x = x + self.dropout_2(self.attn_encoder(x, k_enc, v_enc, src_mask))
        x = self.norm_2(x)
        
        x = x + self.dropout_3(self.ff(x))
        x = self.norm_3(x)
        
        return x

## Test Module

In [75]:
d_model = 16
heads = 4
bs = 2
seq_len = 10

In [76]:
el = EncoderLayer(d_model, heads)

In [77]:
x_src = torch.rand(size=(bs,seq_len, d_model))
x_trg = torch.rand(size=(bs,seq_len, d_model))

In [78]:
enc_output = el(x_src,mask=None)
enc_output.size()

torch.Size([2, 10, 16])

In [81]:
dl = DecoderLayer_basic(d_model, heads)

dec_output = dl(x_trg, enc_output, src_mask=None, trg_mask=None)

dec_output.size()

# Add-Norm Module

The above implementation of `EncoderLayer_basic()`, `DecoderLayer_basic()` ic correct. However we can create an `Add-Norm` module to make the code more modular and get rid of repeatitive code-block

In [89]:
class AddNorm(nn.Module):
    def __init__(self, d_model, dropout_pct = 0.1):
        super().__init__()
        self.norm = Norm(d_model)
        self.dropout = nn.Dropout(dropout_pct)
        
    def forward(self, x, attn_output):
        # add
        x = x + self.dropout(attn_output)
        
        # normalize
        x = self.norm(x)
        
        return x

# Improved Encode Layer with AddNorm module

In [90]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout_pct = 0.1):
        super().__init__()
        
        self.attn = MultiHeadAttention(heads, d_model, dropout=dropout_pct)
        self.ff = FeedForward(d_model, dropout_pct = dropout_pct)
        self.add_norm_1 = AddNorm(d_model, dropout_pct)
        self.add_norm_2 = AddNorm(d_model, dropout_pct)
        
    def forward(self, x, mask):
        # q = k = v = x
        x = self.add_norm_1(x,self.attn(x,x,x,mask))
        x = self.add_norm_2(x, self.ff(x))
        return x

## Test Module

In [91]:
d_model = 16
heads = 4
bs = 2
seq_len = 10

el_1 = EncoderLayer(d_model, heads)

x = torch.rand(size=(bs,seq_len, d_model))

output = el_1(x,mask=None)
output.size()

torch.Size([2, 10, 16])

In [94]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout_pct = 0.1):
        super().__init__()
        
        self.attn_decoder = MultiHeadAttention(heads, d_model)
        self.attn_encoder = MultiHeadAttention(heads, d_model)
        
        self.ff = FeedForward(d_model)
        
        self.add_norm_1 = AddNorm(d_model, dropout_pct)
        self.add_norm_2 = AddNorm(d_model, dropout_pct)
        self.add_norm_3 = AddNorm(d_model, dropout_pct)
        
        
        
    def forward(self, x, encoder_output, src_mask, trg_mask):
        
        """
        x: this x comes from the target language
        """
        
        q = k = v = x
        x = self.add_norm_1(x ,self.attn_decoder(q,k,v,trg_mask))
        
        k_enc = v_enc = encoder_output
        x = self.add_norm_2(x, self.attn_encoder(x, k_enc, v_enc, src_mask))
        
        x = self.add_norm_3(x, self.ff(x))
        
        return x

## Test Module

In [95]:
d_model = 16
heads = 4
bs = 2
seq_len = 10

el = EncoderLayer(d_model, heads)
x_src = torch.rand(size=(bs,seq_len, d_model))
enc_output = el(x_src,mask=None)


dl = DecoderLayer(d_model, heads)
x_trg = torch.rand(size=(bs,seq_len, d_model))
dec_output = dl(x_trg, enc_output, src_mask=None, trg_mask=None)

dec_output.size()

torch.Size([2, 10, 16])

# Replication of EncoderLayer and DecoderLayer

One last Variable: If you look at the diagram closely you can see a `Nx` next to the encoder and decoder architectures. In reality, the encoder and decoder in the diagram above represent one layer of an encoder and one of the decoder. `N` is the variable for the number of layers there will be. Eg. if `N=6`, the data goes through six encoder layers (with the architecture seen above), then these outputs are passed to the decoder which also consists of six repeating decoder layers.

In [96]:
import copy

In [97]:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [101]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_encoder_layer, heads, dropout):
        super().__init__()
        self.n_encoder_layer = n_encoder_layer
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEmbedding(d_model)
        enc_layer = EncoderLayer(d_model, heads, dropout)
        # stack enc_layers
        self.enc_layers = get_clones(enc_layer, n_encoder_layer)
        
    def forward(self, src_token, mask):
        x = self.embed(src_token)
        x = self.pe(x)
        
        for i in range(self.n_encoder_layer):
            x = self.enc_layers[i](x, mask)
        
        # the EncoderLayer return normalized x
        # so no need to pass through Norm() again
        
        return x

In [102]:
d_model = 16
heads = 4
bs = 2
seq_len = 10
vocab_size = 2000

enc = Encoder(vocab_size, d_model, 
              n_encoder_layer=6, heads=heads, dropout=0.1)

In [111]:
x_src = torch.tensor([1,2,3,4,5, 6,7, 8, 9, 10])
enc_output = enc(x_src,mask=None)

In [112]:
enc_output.size()

torch.Size([10, 10, 16])